In [1]:
# Importing dependencies
import os
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# Path to JSON
businessJSON = os.path.join('sourceData', 'business.json')

In [3]:
# Creating pd dataframe
business = pd.read_json(businessJSON, lines=True)

In [4]:
# Select only the businesses in Ontario
business_on = business.loc[business['state'] == 'ON']

In [5]:
# Dropping any rows with blank values in these categories
business_on_clean = business_on.dropna(subset=['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude', 'attributes',
                                                          'categories', 'hours']).reset_index(drop=True)

In [6]:
# Selecting all of the restaurants
restaurant = business_on_clean[business_on_clean['categories'].str.contains('Restaurants')].reset_index(drop=True)

In [7]:
# Regex to fix spelling mistakes 
restaurant = restaurant.replace({'city': {'^AGINCOURT$': 'Agincourt',
                                            '^Bradford West Gwillimbury$': 'Bradford',
                                            '^East Ajax$': 'Ajax',
                                            '^Caledon.{,8}$': 'Caledon',
                                            '^East Gwil{1,2}imbury$': 'East Gwillimbury',
                                            '(?i)^.*icoke$': 'Etobicoke',
                                            '^.{,9}Toro?nto.{,9}$': 'Toronto',
                                            'Malton': 'Mississauga',
                                            '^.{,5}Missis{1,2}a?ua?g.{1,2}$': 'Mississauga',
                                            '^Regional Municipality of York$': 'North York',
                                            '(?i)^North.{0,2}York$': 'North York',
                                            '^York Regional Municipality$': 'York',
                                            '^Willowdale$': 'North York',
                                            '^North of Brampton$': 'Brampton',
                                            '(?i)^Oak.?ridges$': 'Oak Ridges',
                                            '^oakville$': 'Oakville',
                                            '(?i)^Richmond?.?Hill?$': 'Richmond Hill',
                                            '^.{,8}Scar.?bo?rough$': 'Scarborough',
                                            '^.{,11}Stouffville$': 'Stouffville',
                                            '(?i)^Thornhil{,2}$': 'Thornhill',
                                            '^.*Vaugh.{,3}$': 'Vaughan',
                                            '^Wh.?i.?by$': 'Whitby'}}, regex=True)

In [8]:
# Only taking these columns
restaurant = restaurant.loc[:, ['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude','categories', 'stars', 'hours', 'attributes']]
restaurant.columns = ['Name', 'Address', 'Postal_code', 'City', 'Province', 'Latitude', 'Longitude', 'Categories', 'Stars', 'Hours', 'Attributes']

In [9]:
hours_raw = pd.DataFrame(json_normalize(data=restaurant['Hours']))

In [10]:
# Reorganise columns
hours_raw = hours_raw.loc[:,['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
hours_raw.tail()

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
10712,0:0-0:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-23:0,12:0-23:0,12:0-23:0
10713,11:30-22:30,11:30-22:30,11:30-22:30,11:30-23:0,11:30-23:0,11:30-23:0,11:30-22:30
10714,11:30-22:0,NaN,11:30-22:0,11:30-22:0,11:30-23:45,12:0-23:45,12:0-22:0
10715,0:0-0:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,12:0-23:0,12:0-22:0
10716,11:0-21:30,11:0-21:30,11:0-21:30,11:0-21:30,11:0-22:30,11:0-22:30,16:0-21:30


In [11]:
# Create a new DF with opening and closing hours
columns=hours_raw.columns
hours = hours_raw

for column in columns:
    hours[[f"{column}_open", f"{column}_close"]]=hours_raw[column].str.split('-', expand=True)
#     print(column)
hours.drop(columns=columns, inplace=True)

In [12]:
# Add hours column to the original DF
restaurant = restaurant.join(hours)
restaurant.drop(columns='Hours', inplace=True)
restaurant.tail()

,Name,Address,Postal_code,City,Province,Latitude,Longitude,Categories,Stars,Attributes,...,Wednesday_open,Wednesday_close,Thursday_open,Thursday_close,Friday_open,Friday_close,Saturday_open,Saturday_close,Sunday_open,Sunday_close
10712,The King's Kitchen,"9275 Hwy 48, Unit 11",L6E 1A2,Markham,ON,43.893994,-79.263055,"Imported Food, Chinese, Food, Seafood, Special...",3.0,"{'Caters': 'True', 'RestaurantsGoodForGroups':...",...,11:0,22:0,11:0,22:0,11:0,23:0,12:0,23:0,12:0,23:0
10713,KOKO! Share Bar,81 Yorkville Avenue,M5R 1C1,Toronto,ON,43.670948,-79.391502,"Korean, Japanese, Asian Fusion, Restaurants",3.5,"{'RestaurantsTakeOut': 'True', 'Alcohol': 'u'f...",...,11:30,22:30,11:30,23:0,11:30,23:0,11:30,23:0,11:30,22:30
10714,Indian Hero,8920 Highway 50,L6P 3A3,Brampton,ON,43.775089,-79.653807,"Restaurants, Indian",3.0,"{'WiFi': ''free'', 'RestaurantsTakeOut': 'True...",...,11:30,22:0,11:30,22:0,11:30,23:45,12:0,23:45,12:0,22:0
10715,Thai Fantasy,578 Yonge Street,M4Y 1Z3,Toronto,ON,43.665120,-79.384809,"Restaurants, Thai",4.0,"{'RestaurantsPriceRange2': '2', 'RestaurantsGo...",...,11:0,23:0,11:0,23:0,11:0,23:0,12:0,23:0,12:0,22:0
10716,Asia Hut,1450 Kingston Rd,L1V 1C1,Pickering,ON,43.841844,-79.083881,"Restaurants, Soup, Chinese, Caribbean",4.5,"{'BikeParking': 'True', 'RestaurantsAttire': '...",...,11:0,21:30,11:0,21:30,11:0,22:30,11:0,22:30,16:0,21:30


In [16]:
categories = restaurant['Categories'].str.split(',', expand=True)
categories.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
10712,Imported Food,Chinese,Food,Seafood,Specialty Food,Thai,Ethnic Food,Restaurants,None,None,None,None,None,None,None,None,None,None
10713,Korean,Japanese,Asian Fusion,Restaurants,None,None,None,None,None,None,None,None,None,None,None,None,None,None
10714,Restaurants,Indian,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
10715,Restaurants,Thai,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
10716,Restaurants,Soup,Chinese,Caribbean,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [27]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

import numpy as np

In [33]:
categories_unique=explode(restaurant.assign(var1=restaurant.Categories.str.split(',')), 'var1')
categories_unique['var1'].unique().shape

(611,)